In [1]:
import pandas as pd
import xarray as xr
import datetime
import os
import numpy as np
import matplotlib.pyplot as plt
import time

from dirs import coorddir, wrfdir, prepdir
from myutils import metaread_veg

In [2]:
domain = "d02"

lat_wrf, lon_wrf, z_wrf, vgtyp_wrf, vegfra_wrf = metaread_veg(coorddir, domain)

lat_wrf = xr.DataArray(lat_wrf, dims=["lat2d", "lon2d"])
lon_wrf = xr.DataArray(lon_wrf, dims=["lat2d", "lon2d"])
z_wrf = xr.DataArray(z_wrf, dims=["lat2d", "lon2d"])
vgtyp_wrf = xr.DataArray(vgtyp_wrf, dims=["lat2d", "lon2d"]) 
vegfra_wrf = xr.DataArray(vegfra_wrf, dims=["lat2d", "lon2d"])

## Compute slope and aspect
myslopx, myslopy = np.gradient(z_wrf, 9000)
slope_wrf = np.degrees(np.arctan(np.sqrt(myslopx**2 + myslopy**2)))
aspect_wrf = np.degrees(np.arctan2(-myslopy,myslopx))
## Convert aspect to compass direction (clockwise from north)
aspect_q2 = (aspect_wrf > 90) & (aspect_wrf <= 180) ## [90, 180]
aspect_wrf = 90.0 - aspect_wrf
aspect_wrf[aspect_q2] = 360.0 + aspect_wrf[aspect_q2]


coords = {'lat': (['lat2d','lon2d'], lat_wrf), \
                 'lon': (['lat2d','lon2d'], lon_wrf) }

gcms = ['cesm2','mpi-esm1-2-lr','cnrm-esm2-1',
        'ec-earth3-veg','fgoals-g3','ukesm1-0-ll',
        'canesm5','access-cm2','ec-earth3',]


variants = ['r11i1p1f1','r7i1p1f1','r1i1p1f2',
            'r1i1p1f1','r1i1p1f1','r2i1p1f2',
            'r1i1p2f1','r5i1p1f1','r1i1p1f1',]
   

calendar = ['365_day','proleptic_gregorian','proleptic_gregorian',
            'proleptic_gregorian','365_day','360_day',
             '365_day','proleptic_gregorian', 'proleptic_gregorian',]


ssps = ['ssp370','ssp370','ssp370','ssp370',
        'ssp370','ssp370','ssp370','ssp370',
        'ssp370']


In [15]:
sweBC = xr.open_dataarray('/global/cfs/cdirs/m4099/fate-of-snotel/wrfdata/snowmaxBC.nc')
swe_apr1_corrupted = xr.zeros_like(sweBC)

In [4]:
def get_gcm_filepath (gcm_ind, var, year):
    if year < 2014:
        return f'{wrfdir}{gcms[gcm_ind]}_{variants[gcm_ind]}_historical_bc/postprocess/d02/' + f'{var}.daily.{gcms[gcm_ind]}_hist_BIAS_CORRECT_{variants[gcm_ind]}_d02_{str(year)}.nc'
    else:
        return f'{wrfdir}{gcms[gcm_ind]}_{variants[gcm_ind]}_ssp370_bc/postprocess/d02/' + f'{var}.daily.{gcms[gcm_ind]}_ssp370_BIAS_CORRECT_{variants[gcm_ind]}_d02_{str(year)}.nc'

## Sample SWE data around Apr 1, with a standard deviation of 7 days.

In [131]:
start = time.time()

var_name='snow'
for gcm_ind in range(9):
    rng = np.random.default_rng(seed=0) ## This way, the offset days will be consistent across gcm's
    gcm = f'{gcms[gcm_ind]}_{variants[gcm_ind]}_ssp370'
    
    for year in range(1980,2100):
        fname = get_gcm_filepath(gcm_ind, var_name, year)
        gcm_dataset = xr.open_dataarray(fname)
        date_list = list(gcm_dataset.day.values.astype('int').astype('str'))
        apr1_date = f'{year+1}0401'
        offset_date = date_list[date_list.index(apr1_date) + int(np.around(rng.normal(0, 7)))]
        
        # offset_date = (pd.Timestamp(apr1_date) + pd.Timedelta(np.round(rng.normal(0, 7)), 'd')).strftime('%Y%m%d')
        
        swe_apr1_corrupted.loc[dict(gcm=gcm, time=year+1)] = gcm_dataset.sel(day=offset_date)

end = time.time()
print (f'Time: {end-start} seconds')

swe_apr1_corrupted.to_netcdf(f'{prepdir}swe_apr1_SynthErr.nc')

Time: 87.93744230270386 seconds
